In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# d: 수요지 i에서 공급지 j까지의 거리
d = np.array([
    [1, 185, 392, 225, 268, 583, 455, 568],
    [185, 1, 232, 163, 295, 404, 366, 491],
    [392, 232, 1, 256, 400, 285, 370, 375],
    [225, 163, 256, 1, 151, 325, 220, 339],
    [268, 295, 400, 151, 1, 521, 302, 414],
    [583, 404, 285, 325, 521, 1, 215, 234],
    [455, 366, 370, 220, 302, 215, 1, 119],
    [568, 491, 375, 339, 414, 234, 119, 1]])

# h: 수요지의 수요
h = [5983, 5232, 5983, 14575, 12561, 2885, 13698, 594]

# 기존 카페 수 반영
w = [1, 0, 0, 2, 0, 1, 0, 1]

num_i = len(w)
num_j = len(w)

# 모델 생성
model = gp.Model("cafe_optimization")

# 변수
x = model.addVars(num_j, vtype=GRB.BINARY, name="x")
y = model.addVars(num_i, num_j, vtype=GRB.CONTINUOUS, name="y")
z = model.addVars(num_i, vtype=GRB.CONTINUOUS, name="z")

# 목적 함수 설정
objective = gp.LinExpr()
for i in range(num_i):
    for j in range(num_j):
        if j in [0, 5, 7]:
            objective += (1/2) * y[i, j] * x[j]
        elif j == 3:
            objective += (1/3) * y[i, j] * x[j]
        else:
            objective += y[i, j] * x[j]
model.setObjective(objective, GRB.MAXIMIZE)

# 제약 조건 설정
# Constraint 1 - 하나의 새로운 카페만 열어야 함
model.addConstr(gp.quicksum(x[j] for j in range(num_j)) == 1, "cafe_count")

# Constraint 2 - 각 수요지의 수요가 충족되어야 함
for i in range(num_i):
    model.addConstr(gp.quicksum(y[i, j] for j in range(num_j)) == h[i], f"demand_{i}")

# Constraint 3 - 보조변수 z 정의
for i in range(num_i):
    model.addConstr(z[i] * gp.quicksum((1 / (d[i, j]**2)) * x[j] for j in range(num_j)) == 1)

# Constraint 4 - y_ij 정의
for i in range(num_i):
    for j in range(num_j):
        # 분모 계산
        denominator = gp.LinExpr()
        denominator += (1 / (d[i, 0]**2))
        denominator += (1 / (d[i, 3]**2))
        denominator += (1 / (d[i, 5]**2))
        denominator += (1 / (d[i, 7]**2))
        
        #분자 계산
        top = w[j] / (d[i, j]**2) + x[j] / (d[i, j]**2)
        
        # y_ij 제약식 추가
        model.addConstr(y[i, j] * (denominator + z[i]) == top * h[i], name=f"y_ij_{i}_{j}")

# 모델 최적화
model.optimize()

# 결과 출력
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for j in range(num_j):
        if x[j].x == 1:
            print(f"New cafe will be opened at location {j+1}")
else:
    print("No optimal solution found.")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-28
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 80 columns and 72 nonzeros
Model fingerprint: 0x33abc81d
Model has 64 quadratic objective terms
Model has 72 quadratic constraints
Variable types: 72 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e-06, 1e+00]
  QLMatrix range   [4e-05, 1e+04]
  Objective range  [0e+00, 0e+00]
  QObjective range [7e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
  QRHS range       [2e-03, 3e+04]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)